# The Necessery Dependencies

Install the required, you can use the link below to check the steps of installation.
https://github.com/ShashankAthreya/face_recognition_OFD

In [ ]:
import face_recognition
import os
import cv2
import numpy as np
from PIL import Image, ImageDraw
from IPython.display import display
import pandas as pd
import csv
import pickle

### The necessary varaibles are defined below

In [ ]:
Training_Data = "Training_Data"
Tolerence = 0.01
Frame_thickness = 3
Font_thickness = 2
Model = "cnn" #or hog
new_known_faces = []
new_known_names = []

### The pre-existing model is loaded using the below lines of code

In [ ]:
with open("known_faces.txt", "rb") as fp:
    imported_known_faces = pickle.load(fp)
with open("known_names.txt", "rb") as fp:
    imported_known_names = pickle.load(fp)

### Adding additional datasets

Place the dataset in the Training_Data folder and execute the below lines of code

In [ ]:
count = 0
print("Initialising new tests")
for name in os.listdir(Training_Data):
    for filename in os.listdir(f"{Training_Data}/{name}"):
        image = face_recognition.load_image_file(f"{Training_Data}/{name}/{filename}")
        #print(f"{Training_Data}/{name}/{filename}")
        try:
            encoding = face_recognition.face_encodings(image)[0]
        except IndexError:
            pass
        new_known_faces.append(encoding)
        new_known_names.append(name)
    count += 1
    print(count,name)
    
joined_known_faces = imported_known_faces + new_known_faces
joined_known_names = imported_known_names + new_known_names

print("Adding Newly Tainied Names")

with open("known_faces.txt", "wb") as fp:
    pickle.dump(joined_known_faces, fp)
with open("known_names.txt", "wb") as fp:
    pickle.dump(joined_known_names, fp)
    
with open("known_faces.txt", "rb") as fp:
    imported_known_faces = pickle.load(fp)
with open("known_names.txt", "rb") as fp:
    imported_known_names = pickle.load(fp)
    
print("Faces have been added")

### Testing the trained model
Execute the below lines of code to test the trained model.
Add the path of the file in line 3.

In [ ]:
print("Starting Testing for unknown faces")

unknown_image = face_recognition.load_image_file("test_image.jpg")#Add the file path here


face_locations = face_recognition.face_locations(unknown_image)
face_encodings = face_recognition.face_encodings(unknown_image, face_locations)
pil_image = Image.fromarray(unknown_image)
draw = ImageDraw.Draw(pil_image)
for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
    matches = face_recognition.compare_faces(imported_known_faces, face_encoding, Tolerence)
    name = "Unknown"
    face_distances = face_recognition.face_distance(imported_known_faces, face_encoding)
    best_match_index = np.argmin(face_distances)
    if matches[best_match_index]:
        name = imported_known_names[best_match_index]
    draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))
    text_width, text_height = draw.textsize(name)
    draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
    draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

del draw
display(pil_image)
print("Confidence level ",(1-Tolerence)*100,"%")

### Additional operations
print to check if your trained name has been added

In [ ]:
print(imported_known_names)